In [9]:
from dotenv import load_dotenv
import os
from openai import OpenAI
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
# Load the test set 
test_df = pd.read_csv("C:/Users/Vasilis/Desktop/MSc in Data Science/Dissertation/Final/Hard Mining/20-06-2025/test_set.csv")

In [4]:
# Set the OpenAI API key
load_dotenv()
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

def create_prompt(title1, title2):
    return f"""Do the following two product titles refer to the same product? Answer only with "Yes" or "No".

'''Title 1: {title1}'''
'''Title 2: {title2}'''
Answer:"""

def gpt4_match(title1, title2):
    prompt = create_prompt(title1, title2)
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}],
        temperature=0
    )
    answer = response.choices[0].message.content.lower()
    return 1 if "yes" in answer else 0


# Run inference on test DataFrame
predictions = []
for idx, row in test_df.iterrows():
    pred = gpt4_match(row["title_1"], row["title_2"])
    print(f"Pair {idx}: Prediction: {'Match' if pred == 1 else 'No Match'}")
    predictions.append(pred)

Pair 0: Prediction: No Match
Pair 1: Prediction: Match
Pair 2: Prediction: No Match
Pair 3: Prediction: Match
Pair 4: Prediction: No Match
Pair 5: Prediction: Match
Pair 6: Prediction: No Match
Pair 7: Prediction: No Match
Pair 8: Prediction: No Match
Pair 9: Prediction: No Match
Pair 10: Prediction: No Match
Pair 11: Prediction: Match
Pair 12: Prediction: No Match
Pair 13: Prediction: No Match
Pair 14: Prediction: No Match
Pair 15: Prediction: Match
Pair 16: Prediction: Match
Pair 17: Prediction: Match
Pair 18: Prediction: Match
Pair 19: Prediction: No Match
Pair 20: Prediction: Match
Pair 21: Prediction: No Match
Pair 22: Prediction: No Match
Pair 23: Prediction: Match
Pair 24: Prediction: No Match
Pair 25: Prediction: No Match
Pair 26: Prediction: No Match
Pair 27: Prediction: No Match
Pair 28: Prediction: Match
Pair 29: Prediction: No Match
Pair 30: Prediction: No Match
Pair 31: Prediction: Match
Pair 32: Prediction: Match
Pair 33: Prediction: No Match
Pair 34: Prediction: No Match

In [5]:
print(classification_report(test_df["label"].values, predictions, digits=4))

              precision    recall  f1-score   support

           0     0.8146    0.9781    0.8889       732
           1     0.9517    0.6590    0.7787       478

    accuracy                         0.8521      1210
   macro avg     0.8831    0.8186    0.8338      1210
weighted avg     0.8687    0.8521    0.8454      1210



In [7]:
test_df["predicted_label"] = predictions  
test_df["is_correct"] = test_df["label"] == test_df["predicted_label"]

In [10]:
y_true = test_df["label"]
y_pred = test_df["predicted_label"]

cm = confusion_matrix(y_true, y_pred)

tn, fp, fn, tp = cm.ravel()

print(f"True Negatives: {tn}")
print(f"False Positives: {fp}")
print(f"False Negatives: {fn}")
print(f"True Positives: {tp}")

True Negatives: 716
False Positives: 16
False Negatives: 163
True Positives: 315


In [11]:
# Type I Error
false_positives = test_df[(test_df["label"] == 0) & (test_df["predicted_label"] == 1)]

# Type II Error
false_negatives = test_df[(test_df["label"] == 1) & (test_df["predicted_label"] == 0)]

In [12]:
false_positives[["title_1", "title_2"]].head(10)

,title_1,title_2
49,samsung galaxy a5 sm a510f sim free smartphone...,samsung galaxy a5 sim free smartphone white
143,motorola mobile phone in indigo,motorola moto g6 smartphone deep indigo
161,nokia 3310 2017 gelb,nokia 3310 retro dual sim handy gelb
313,samsung galaxy s8 64gb orchid grey,samsung galaxy s8 64 gb orchid grey
317,nokia 3310 mobile phone in warm red,nokia 3310 3g warm red
336,samsung b2710 solid immerse mobile phone,samsung b2710 solid immerse black
340,htc one x pj46100 16gb grey unlocked smartphon...,htc one x grey
436,samsung galaxy s9 plus dual sim 64gb midnight ...,samsung galaxy s9 64 gb dual sim midnight blac...
472,sony xperia xa2 ultra silver,sony xperia xa2 ultra dual sim silver
516,sony xperia m4 aqua,sony m4 aqua silver


In [13]:
false_negatives[["title_1", "title_2"]].head(10)

,title_1,title_2
13,xiaomi redmi note 5 smartphone 5.99 2160 x,redmi mzb6122eu smartphone 15.2 cm 5.99 4 gb 6...
19,sim free 5030 white by doro,doro phoneeasy 5030 1.7 78g black graphite
30,lg k10 2017 black 5.3 16gb 4g unlocked sim free,lg k10 2017 m250n 5.3 single sim 4g 2gb 16gb 2...
33,huawei p20 lite 64gb midnight black,huawei p20 lite dual sim smartphone blau
40,bea fon c140 schwarz t rkis,beafon classic line c140 handy dual sim schwar...
42,apple iphone 6s 32gb grau handy,apple iphone 6s 32 gb silver
43,apple iphone 7 plus 32gb black,apple iphone 7 plus
44,doro phoneeasy 632 sim free smartphone black,diamond lamps vlt xd280lp 230w projector lamp
50,nokia 216 sim free feature phone blue,nokia nokia 216 grey a00028037 ad01
52,nokia 11ne1r01a03 3 copper dual sim unlocked,nokia 3 copper white dual sim unlocked


In [14]:
out_metrics_dir = config.METRICS_DIR / "product_matching" / "gpt4_zeroshot"
out_metrics_dir.mkdir(parents=True, exist_ok=True)

false_positives.to_csv(out_metrics_dir / "false_positives.csv", index=False)
false_negatives.to_csv(out_metrics_dir / "false_negatives.csv", index=False)